# Segmenting and Clustering Part 1

Begin by importing all of the required libraries

In [1]:
# Begin by importing all of the required libraries
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import json
import requests

from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

Scrape data from Wikipedia with requests

In [2]:
# Scrape data from Wikipedia with requests
raw_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Use the BeautifulSoup package to parse the df and create an object

In [3]:
# Use the BeautifulSoup package to parse the df and create an object:
soup = BeautifulSoup(raw_data, 'lxml')

Create the Table

In [4]:
# Create the table
table = soup.find('table')
table_rows = table.tbody.find_all('tr')
table_contents=[]

Write a loop using the given code to process cells with Borough "assigned", combine the duplicate entry of M5A, and match Boroughs and Neighborhoods when a cell has a Borough but no Neighborhood.

In [5]:
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


Print the resulting dataframe:

In [6]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Use Groupby to group the Neighborhoods by Borough and print the resulting df:df.head()

In [7]:
# Group the neighborhoods by borough:
df = df.groupby(['PostalCode', 'Borough'], as_index= False).agg(lambda x: ','.join(x))
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now that the dataframe has been tidied, let's create a "Final" dataframe for our assignment:

In [8]:
# Based on the dataframe created above, reassign rows and columns to match the required dataframe:
final_columns = ['PostalCode', 'Borough', 'Neighborhood']
final_df = pd.DataFrame(columns = final_columns)
final_list = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']

# Write a for loop to populate the final dataframe based on the PostalCode:
for postcode in final_list:
    final_df = final_df.append(df[df['PostalCode'] == postcode], ignore_index = True)

In [9]:
# Print the final dataframe to ensure it matches that of the assignment
final_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


Print the shape of the dataframe:

In [10]:
# Print the shape of the dataframe:
df.shape

(103, 3)